# Interactive Data Merge Tool

**Just 2 cells to run:**
1. **Cell 1** - Setup (run once)
2. **Cell 2** - Interactive menu (type commands, see results)

No code to edit. Everything is done through typed commands.

In [ ]:
# ================================================================
# CELL 1: Setup - Run this once (Shift+Enter), then move to Cell 2
# ================================================================

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MaxNLocator
from pathlib import Path
from IPython.display import clear_output, display
import sys, copy
from datetime import datetime

SCRIPT_DIR = Path().resolve()
if str(SCRIPT_DIR) not in sys.path:
    sys.path.insert(0, str(SCRIPT_DIR))
from aat_data_loader_multisweep import AATDataLoader

%matplotlib inline

plt.rcParams.update({
    'figure.facecolor': 'white', 'axes.facecolor': 'white',
    'axes.linewidth': 1.5, 'axes.labelweight': 'bold', 'axes.labelsize': 14,
    'xtick.labelsize': 12, 'ytick.labelsize': 12, 'font.weight': 'bold',
    'legend.frameon': True, 'legend.framealpha': 0.9, 'legend.fontsize': 11,
})

PALETTES = {
    'muted': ['#CC6677','#332288','#DDCC77','#117733','#88CCEE','#882255','#44AA99','#999933','#AA4499'],
    'okabe': ['#E69F00','#56B4E9','#009E73','#F0E442','#0072B2','#D55E00','#CC79A7','#000000'],
    'vibrant': ['#EE7733','#0077BB','#33BBEE','#EE3377','#CC3311','#009988','#BBBBBB'],
}


class DataMerger:
    """Interactive data merging tool."""

    def __init__(self):
        self.loader = AATDataLoader()
        self.raw = {}           # label -> [measurements]
        self.cleaned = {}       # label -> [measurements]
        self.merged = []        # final merged list
        self.merge_sources = [] # which label each merged sweep came from
        self.palette = 'muted'
        self.scale = 'nA'
        self.title = 'Merged Plot'
        self.legends = None
        self.annotations = []
        self.show_retrace = True
        self.xlim = None
        self.ylim = None
        self._file_counter = 0

    # ── Helpers ──────────────────────────────────────────────────

    def _ask(self, prompt, default=None):
        val = input(prompt).strip()
        return val if val else default

    def _ask_int(self, prompt, default=None):
        val = self._ask(prompt, str(default) if default is not None else None)
        try:
            return int(val) if val else default
        except ValueError:
            return default

    def _ask_float(self, prompt, default=None):
        val = self._ask(prompt, str(default) if default is not None else None)
        try:
            return float(val) if val else default
        except ValueError:
            return default

    def _ask_yn(self, prompt, default=True):
        hint = 'Y/n' if default else 'y/N'
        val = self._ask(f'{prompt} [{hint}]: ')
        if not val:
            return default
        return val.lower().startswith('y')

    def _colors(self):
        return PALETTES.get(self.palette, PALETTES['muted'])

    def _scale_factor(self):
        return 1e9 if self.scale == 'nA' else 1e6

    def _unit(self):
        return 'nA' if self.scale == 'nA' else r'$\mu$A'

    def _unit_plain(self):
        return 'nA' if self.scale == 'nA' else 'uA'

    def _get_data(self, label):
        """Get cleaned data if available, otherwise raw."""
        return self.cleaned.get(label, self.raw.get(label, []))

    def _print_sweep_table(self, measurements, label=''):
        prefix = f'{label} | ' if label else ''
        for i, m in enumerate(measurements):
            vg = m['forward']['Vg']
            bwd = 'Yes' if m['backward'] else 'No'
            print(f'  {prefix}Sweep {i}: Vd={m["Vd"]:.2f}V | '
                  f'Vg=[{vg[0]:.1f} to {vg[-1]:.1f}]V | '
                  f'Retrace={bwd} | Points={len(vg)}')

    def _pick_label(self, prompt='Select file'):
        labels = list(self.raw.keys())
        if not labels:
            print('No files loaded yet!')
            return None
        print(f'\nLoaded files:')
        for i, l in enumerate(labels):
            status = ' [cleaned]' if l in self.cleaned else ''
            print(f'  {i+1}. {l}{status}')
        choice = self._ask(f'{prompt} (number or name): ')
        if not choice:
            return None
        try:
            idx = int(choice) - 1
            if 0 <= idx < len(labels):
                return labels[idx]
        except ValueError:
            if choice in labels:
                return choice
        print(f'Invalid selection: {choice}')
        return None

    # ── Core Actions ─────────────────────────────────────────────

    def load(self):
        """Load a data file."""
        path = self._ask('\nFile path (drag & drop or paste): ')
        if not path:
            return
        # Remove quotes if drag-dropped
        path = path.strip().strip("'\"")
        p = Path(path)
        if not p.exists():
            print(f'File not found: {path}')
            return

        self._file_counter += 1
        default_label = f'File_{self._file_counter}'
        label = self._ask(f'Label for this file [{default_label}]: ', default_label)

        if p.is_dir():
            measurements = self.loader.load_directory(p)
        else:
            measurements = self.loader.load_measurement(p)

        if measurements:
            self.raw[label] = measurements
            print(f'\nLoaded "{label}": {len(measurements)} sweep(s) from {p.name}')
            self._print_sweep_table(measurements)
        else:
            print('Failed to load data.')
            self._file_counter -= 1

    def status(self):
        """Show all loaded files and their status."""
        if not self.raw:
            print('\nNo files loaded yet. Use "load" to add files.')
            return
        print(f'\n{"="*60}')
        print(f'{"Label":<15} {"Sweeps":<10} {"Status":<15} {"Retrace"}')
        print(f'{"-"*60}')
        for label, data in self.raw.items():
            n_raw = len(data)
            if label in self.cleaned:
                n_clean = len(self.cleaned[label])
                status = f'Cleaned ({n_clean}/{n_raw})'
            else:
                status = 'Raw'
            has_bwd = any(m['backward'] for m in self._get_data(label))
            print(f'{label:<15} {n_raw:<10} {status:<15} {"Yes" if has_bwd else "No"}')
        print(f'{"="*60}')

    def preview(self):
        """Preview a file's sweeps."""
        label = self._pick_label('Preview which file')
        if not label:
            return

        data = self._get_data(label)
        is_cleaned = label in self.cleaned
        colors = self._colors()
        sf = self._scale_factor()

        fig, ax = plt.subplots(1, 1, figsize=(10, 7))
        for i, m in enumerate(data):
            c = colors[i % len(colors)]
            ax.plot(m['forward']['Vg'], m['forward']['Id']*sf, '-', color=c,
                    linewidth=2.5, label=f'Sweep {i}: Vd={m["Vd"]:.1f}V (fwd)')
            if m['backward']:
                ax.plot(m['backward']['Vg'], m['backward']['Id']*sf, '--', color=c,
                        linewidth=1.5, alpha=0.4, label=f'Sweep {i}: Vd={m["Vd"]:.1f}V (bwd)')
        tag = 'CLEANED' if is_cleaned else 'RAW'
        ax.set_title(f'{label} [{tag}] - {len(data)} sweep(s)', fontweight='bold')
        ax.set_xlabel('$V_g$ (V)'); ax.set_ylabel(f'$I_d$ ({self._unit()})')
        ax.legend(fontsize=9, loc='best')
        for lbl in ax.get_xticklabels() + ax.get_yticklabels():
            lbl.set_fontweight('bold')
        plt.tight_layout()
        plt.show()
        self._print_sweep_table(data, label)

    def clean(self):
        """Interactively clean a file."""
        label = self._pick_label('Clean which file')
        if not label:
            return

        source = self.raw[label]  # Always clean from raw
        print(f'\n--- Cleaning "{label}" ({len(source)} sweeps) ---')
        self._print_sweep_table(source)

        # 1. Select sweeps
        print(f'\n[Sweep Selection]')
        sel = self._ask(f'Keep which sweeps? (e.g. 0,2,4 or "all") [all]: ', 'all')
        if sel.lower() == 'all':
            keep_idx = list(range(len(source)))
        else:
            try:
                keep_idx = [int(x.strip()) for x in sel.split(',')]
            except ValueError:
                print('Invalid input, keeping all.')
                keep_idx = list(range(len(source)))

        # 2. Remove retrace?
        print(f'\n[Retrace]')
        remove_retrace = not self._ask_yn('Keep retrace (backward sweep)?', default=True)

        # 3. Trim voltage range?
        print(f'\n[Voltage Trim]')
        do_trim = self._ask_yn('Trim Vg range?', default=False)
        vg_min, vg_max = None, None
        if do_trim:
            vg_min = self._ask_float('  Vg min (e.g. -6): ')
            vg_max = self._ask_float('  Vg max (e.g. 0): ')

        # Apply
        cleaned = []
        for i in keep_idx:
            if i < 0 or i >= len(source):
                print(f'  Sweep {i}: SKIPPED (out of range)')
                continue
            m = copy.deepcopy(source[i])

            if remove_retrace:
                m['backward'] = None

            if vg_min is not None or vg_max is not None:
                lo = vg_min if vg_min is not None else -np.inf
                hi = vg_max if vg_max is not None else np.inf
                mask = (m['forward']['Vg'] >= lo) & (m['forward']['Vg'] <= hi)
                m['forward']['Vg'] = m['forward']['Vg'][mask]
                m['forward']['Id'] = m['forward']['Id'][mask]
                if m['backward']:
                    mask_b = (m['backward']['Vg'] >= lo) & (m['backward']['Vg'] <= hi)
                    m['backward']['Vg'] = m['backward']['Vg'][mask_b]
                    m['backward']['Id'] = m['backward']['Id'][mask_b]

            cleaned.append(m)

        self.cleaned[label] = cleaned
        print(f'\nResult: {len(cleaned)} sweep(s) kept')
        self._print_sweep_table(cleaned, label)

        # Show before/after
        if self._ask_yn('\nShow before/after comparison?', default=True):
            self._compare_plot(label)

    def _compare_plot(self, label):
        """Show before/after comparison."""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        colors = self._colors()
        sf = self._scale_factor()

        for i, m in enumerate(self.raw[label]):
            c = colors[i % len(colors)]
            ax1.plot(m['forward']['Vg'], m['forward']['Id']*sf, '-', color=c,
                    linewidth=2, label=f'S{i}: Vd={m["Vd"]:.1f}V')
            if m['backward']:
                ax1.plot(m['backward']['Vg'], m['backward']['Id']*sf, '--', color=c, linewidth=1.5, alpha=0.4)
        ax1.set_title(f'{label} - BEFORE', fontweight='bold', color='red')
        ax1.set_xlabel('$V_g$ (V)'); ax1.set_ylabel(f'$I_d$ ({self._unit()})')
        ax1.legend(fontsize=8)

        for i, m in enumerate(self.cleaned[label]):
            c = colors[i % len(colors)]
            ax2.plot(m['forward']['Vg'], m['forward']['Id']*sf, '-', color=c,
                    linewidth=2, label=f'Vd={m["Vd"]:.1f}V')
            if m['backward']:
                ax2.plot(m['backward']['Vg'], m['backward']['Id']*sf, '--', color=c, linewidth=1.5, alpha=0.4)
        ax2.set_title(f'{label} - AFTER', fontweight='bold', color='green')
        ax2.set_xlabel('$V_g$ (V)'); ax2.set_ylabel(f'$I_d$ ({self._unit()})')
        ax2.legend(fontsize=8)

        plt.tight_layout()
        plt.show()

    def undo(self):
        """Undo cleaning for a file (revert to raw)."""
        label = self._pick_label('Undo cleaning for')
        if label and label in self.cleaned:
            del self.cleaned[label]
            print(f'Reverted "{label}" to raw data.')
        elif label:
            print(f'"{label}" has no cleaning to undo.')

    def merge(self):
        """Merge files and plot."""
        if not self.raw:
            print('No files loaded!')
            return

        labels = list(self.raw.keys())
        print(f'\nAvailable files:')
        for i, l in enumerate(labels):
            n = len(self._get_data(l))
            tag = ' [cleaned]' if l in self.cleaned else ''
            print(f'  {i+1}. {l} ({n} sweeps){tag}')

        sel = self._ask(f'\nMerge which files? (e.g. 1,2,3 or "all") [all]: ', 'all')
        if sel.lower() == 'all':
            selected = labels
        else:
            try:
                indices = [int(x.strip())-1 for x in sel.split(',')]
                selected = [labels[i] for i in indices if 0 <= i < len(labels)]
            except (ValueError, IndexError):
                print('Invalid selection, merging all.')
                selected = labels

        self.merged = []
        self.merge_sources = []
        for label in selected:
            data = self._get_data(label)
            for m in data:
                self.merged.append(m)
                self.merge_sources.append(label)

        print(f'\nMerged: {len(self.merged)} sweeps from {len(selected)} file(s)')
        for i, (m, src) in enumerate(zip(self.merged, self.merge_sources)):
            print(f'  Sweep {i}: Vd={m["Vd"]:.1f}V  <-- {src}')

        # Ask for legends
        print(f'\n[Legend Labels]')
        print(f'  Default: auto Vd labels')
        custom = self._ask(f'Custom legends? (comma-separated, or press Enter for auto): ')
        if custom:
            self.legends = [x.strip() for x in custom.split(',')]
        else:
            self.legends = None

        # Title
        self.title = self._ask(f'Plot title [{self.title}]: ', self.title)

        # Scale
        s = self._ask(f'Current scale (nA/uA) [{self.scale}]: ', self.scale)
        if s.lower() in ('na', 'ua'):
            self.scale = s.lower().replace('u', 'u')

        # Retrace
        self.show_retrace = self._ask_yn('Show retrace in plot?', self.show_retrace)

        # Plot
        self.plot()

    def plot(self):
        """Plot the merged data."""
        if not self.merged:
            print('Nothing merged yet. Use "merge" first.')
            return

        colors = self._colors()
        sf = self._scale_factor()

        fig, ax = plt.subplots(1, 1, figsize=(10, 7))

        for i, m in enumerate(self.merged):
            c = colors[i % len(colors)]
            if self.legends and i < len(self.legends):
                label = self.legends[i]
            else:
                label = f'Vd = {m["Vd"]:.1f} V'

            ax.plot(m['forward']['Vg'], m['forward']['Id']*sf, '-', color=c,
                    linewidth=2.5, label=label, marker='o', markersize=3, markevery=5)
            if self.show_retrace and m['backward']:
                ax.plot(m['backward']['Vg'], m['backward']['Id']*sf, '--', color=c,
                        linewidth=2, alpha=0.4, marker='s', markersize=3, markevery=5)

        # Annotations
        for ann in self.annotations:
            ax.text(ann[0], ann[1], ann[2], fontsize=ann[4], color=ann[3],
                    fontweight='bold', ha='center', va='center')

        if self.xlim:
            ax.set_xlim(self.xlim)
        if self.ylim:
            ax.set_ylim(self.ylim)

        ax.set_xlabel('$V_g$ (V)', fontsize=14, fontweight='bold')
        ax.set_ylabel(f'$I_d$ ({self._unit()})', fontsize=14, fontweight='bold')
        ax.set_title(self.title, fontsize=14, fontweight='bold', pad=15)
        ax.legend(loc='best', frameon=True, fontsize=11)
        ax.grid(False)
        ax.xaxis.set_major_locator(MaxNLocator(nbins=8))
        ax.yaxis.set_major_locator(MaxNLocator(nbins=8))
        ax.xaxis.set_minor_locator(AutoMinorLocator(2))
        ax.yaxis.set_minor_locator(AutoMinorLocator(2))
        for lbl in ax.get_xticklabels() + ax.get_yticklabels():
            lbl.set_fontweight('bold')

        plt.tight_layout()
        plt.show()

    def tune(self):
        """Fine-tune plot settings."""
        if not self.merged:
            print('Nothing to tune. Use "merge" first.')
            return

        print(f'\n--- Fine-Tune Settings ---')
        print(f'Current: title="{self.title}", palette={self.palette}, scale={self.scale}')
        print(f'         xlim={self.xlim}, ylim={self.ylim}')
        print(f'         annotations={len(self.annotations)}, retrace={self.show_retrace}')
        print()
        print('What to adjust?')
        print('  1. Title')
        print('  2. Axis range (X)')
        print('  3. Axis range (Y)')
        print('  4. Add annotation')
        print('  5. Clear annotations')
        print('  6. Change palette (muted/okabe/vibrant)')
        print('  7. Change scale (nA/uA)')
        print('  8. Toggle retrace')
        print('  9. Change legends')
        print('  0. Done (re-plot)')

        while True:
            choice = self._ask('\nTune> ')
            if not choice or choice == '0':
                break
            elif choice == '1':
                self.title = self._ask(f'New title [{self.title}]: ', self.title)
            elif choice == '2':
                xmin = self._ask_float('X min (Enter=auto): ')
                xmax = self._ask_float('X max (Enter=auto): ')
                self.xlim = (xmin, xmax) if xmin is not None and xmax is not None else None
            elif choice == '3':
                ymin = self._ask_float('Y min (Enter=auto): ')
                ymax = self._ask_float('Y max (Enter=auto): ')
                self.ylim = (ymin, ymax) if ymin is not None and ymax is not None else None
            elif choice == '4':
                x = self._ask_float('  X position: ')
                y = self._ask_float('  Y position: ')
                text = self._ask('  Text: ')
                color = self._ask('  Color [black]: ', 'black')
                size = self._ask_int('  Font size [11]: ', 11)
                if x is not None and y is not None and text:
                    self.annotations.append((x, y, text, color, size))
                    print(f'  Added annotation: "{text}" at ({x}, {y})')
            elif choice == '5':
                self.annotations = []
                print('Annotations cleared.')
            elif choice == '6':
                p = self._ask('Palette (muted/okabe/vibrant): ')
                if p in PALETTES:
                    self.palette = p
            elif choice == '7':
                s = self._ask('Scale (nA/uA): ')
                if s and s.lower() in ('na', 'ua'):
                    self.scale = s.lower()
            elif choice == '8':
                self.show_retrace = not self.show_retrace
                print(f'Retrace: {"ON" if self.show_retrace else "OFF"}')
            elif choice == '9':
                custom = self._ask('Legends (comma-separated, or "auto"): ')
                if custom and custom.lower() != 'auto':
                    self.legends = [x.strip() for x in custom.split(',')]
                else:
                    self.legends = None
            else:
                print('Invalid choice.')
            print(f'Updated. Enter 0 to re-plot or keep adjusting.')

        self.plot()

    def save_data(self, filepath=None):
        """Save merged data to a text file."""
        if not self.merged:
            print('Nothing to save. Use "merge" first.')
            return None

        if filepath is None:
            name = self._ask('Data filename (without extension) [merged_data]: ', 'merged_data')
            outdir = self._ask('Output directory [.]: ', '.')
            fmt = self._ask('Format (txt/csv) [txt]: ', 'txt')
            filepath = Path(outdir) / f'{name}.{fmt}'

        sep = ',' if str(filepath).endswith('.csv') else '\t'
        sf = self._scale_factor()
        unit = self._unit_plain()

        with open(filepath, 'w') as f:
            # Header
            f.write(f'# Merged Data Export\n')
            f.write(f'# Title: {self.title}\n')
            f.write(f'# Date: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\n')
            f.write(f'# Current scale: {unit}\n')
            f.write(f'# Total sweeps: {len(self.merged)}\n')
            f.write(f'#\n')
            f.write(f'# Sources:\n')
            for i, (m, src) in enumerate(zip(self.merged, self.merge_sources)):
                legend = self.legends[i] if self.legends and i < len(self.legends) else f'Vd={m["Vd"]:.1f}V'
                f.write(f'#   Sweep {i}: {legend} (Vd={m["Vd"]:.2f}V) from {src}\n')
            f.write(f'#\n')

            # Column headers
            cols = ['Sweep', 'Source', 'Legend', 'Vd(V)', 'Direction', f'Vg(V)', f'Id({unit})']
            f.write(sep.join(cols) + '\n')

            # Data rows
            for i, (m, src) in enumerate(zip(self.merged, self.merge_sources)):
                legend = self.legends[i] if self.legends and i < len(self.legends) else f'Vd={m["Vd"]:.1f}V'

                # Forward sweep
                for j in range(len(m['forward']['Vg'])):
                    vg = m['forward']['Vg'][j]
                    Id = m['forward']['Id'][j] * sf
                    row = [str(i), src, legend, f'{m["Vd"]:.2f}', 'forward',
                           f'{vg:.6f}', f'{Id:.6f}']
                    f.write(sep.join(row) + '\n')

                # Backward sweep
                if m['backward']:
                    for j in range(len(m['backward']['Vg'])):
                        vg = m['backward']['Vg'][j]
                        Id = m['backward']['Id'][j] * sf
                        row = [str(i), src, legend, f'{m["Vd"]:.2f}', 'backward',
                               f'{vg:.6f}', f'{Id:.6f}']
                        f.write(sep.join(row) + '\n')

        print(f'Data saved: {Path(filepath).resolve()}')
        print(f'  Format: {"CSV" if sep == "," else "Tab-separated"}')
        print(f'  Sweeps: {len(self.merged)}, Scale: {unit}')
        return filepath

    def export(self):
        """Save the final plot and data file."""
        if not self.merged:
            print('Nothing to export. Use "merge" first.')
            return

        name = self._ask('Filename (without extension) [merged_plot]: ', 'merged_plot')
        outdir = self._ask('Output directory [.]: ', '.')
        fmt = self._ask('Image format (png/svg/pdf/eps) [png]: ', 'png')
        dpi = self._ask_int('DPI (300/600) [300]: ', 300)

        colors = self._colors()
        sf = self._scale_factor()

        fig, ax = plt.subplots(1, 1, figsize=(10, 7))
        for i, m in enumerate(self.merged):
            c = colors[i % len(colors)]
            label = self.legends[i] if self.legends and i < len(self.legends) else f'Vd = {m["Vd"]:.1f} V'
            ax.plot(m['forward']['Vg'], m['forward']['Id']*sf, '-', color=c,
                    linewidth=2.5, label=label, marker='o', markersize=3, markevery=5)
            if self.show_retrace and m['backward']:
                ax.plot(m['backward']['Vg'], m['backward']['Id']*sf, '--', color=c,
                        linewidth=2, alpha=0.4, marker='s', markersize=3, markevery=5)

        for ann in self.annotations:
            ax.text(ann[0], ann[1], ann[2], fontsize=ann[4], color=ann[3],
                    fontweight='bold', ha='center', va='center')

        if self.xlim: ax.set_xlim(self.xlim)
        if self.ylim: ax.set_ylim(self.ylim)

        ax.set_xlabel('$V_g$ (V)', fontsize=14, fontweight='bold')
        ax.set_ylabel(f'$I_d$ ({self._unit()})', fontsize=14, fontweight='bold')
        ax.set_title(self.title, fontsize=14, fontweight='bold', pad=15)
        ax.legend(loc='best', frameon=True, fontsize=11)
        ax.grid(False)
        ax.xaxis.set_major_locator(MaxNLocator(nbins=8))
        ax.yaxis.set_major_locator(MaxNLocator(nbins=8))
        ax.xaxis.set_minor_locator(AutoMinorLocator(2))
        ax.yaxis.set_minor_locator(AutoMinorLocator(2))
        for lbl in ax.get_xticklabels() + ax.get_yticklabels():
            lbl.set_fontweight('bold')
        plt.tight_layout()

        # Save plot
        plot_path = Path(outdir) / f'{name}.{fmt}'
        fig.savefig(plot_path, dpi=dpi, bbox_inches='tight', facecolor='white')
        print(f'\nPlot saved: {plot_path.resolve()}')
        print(f'  Format: {fmt.upper()}, DPI: {dpi}')
        plt.show()

        # Save data file
        data_fmt = self._ask('Data file format (txt/csv/none) [txt]: ', 'txt')
        if data_fmt.lower() != 'none':
            data_path = Path(outdir) / f'{name}.{data_fmt}'
            self.save_data(data_path)

    def remove(self):
        """Remove a loaded file."""
        label = self._pick_label('Remove which file')
        if label:
            del self.raw[label]
            self.cleaned.pop(label, None)
            print(f'Removed "{label}".')

    # ── Main Menu ────────────────────────────────────────────────

    def start(self):
        """Launch interactive menu."""
        COMMANDS = {
            'load':    ('Load a data file',          self.load),
            'status':  ('Show loaded files',         self.status),
            'preview': ('Preview a file',            self.preview),
            'clean':   ('Clean a file',              self.clean),
            'undo':    ('Undo cleaning',             self.undo),
            'merge':   ('Merge files & plot',        self.merge),
            'plot':    ('Re-plot merged data',       self.plot),
            'tune':    ('Fine-tune plot settings',   self.tune),
            'export':  ('Export plot + data file',   self.export),
            'save':    ('Save data file only',       self.save_data),
            'remove':  ('Remove a loaded file',      self.remove),
            'help':    ('Show this menu',            None),
            'quit':    ('Exit',                      None),
        }

        print('\n' + '='*50)
        print('  INTERACTIVE DATA MERGE TOOL')
        print('='*50)
        print('\nCommands:')
        for cmd, (desc, _) in COMMANDS.items():
            print(f'  {cmd:<10} {desc}')
        print()

        while True:
            cmd = self._ask('\n>> ').lower()
            if not cmd:
                continue
            if cmd in ('quit', 'exit', 'q'):
                print('Done!')
                break
            elif cmd == 'help':
                print('\nCommands:')
                for c, (d, _) in COMMANDS.items():
                    print(f'  {c:<10} {d}')
            elif cmd in COMMANDS:
                COMMANDS[cmd][1]()
            else:
                # Fuzzy match
                matches = [c for c in COMMANDS if c.startswith(cmd)]
                if len(matches) == 1:
                    if matches[0] in ('quit', 'help'):
                        continue
                    COMMANDS[matches[0]][1]()
                else:
                    print(f'Unknown command: "{cmd}". Type "help" for options.')


merger = DataMerger()
print('Setup complete! Run the next cell to start.')

In [ ]:
# ================================================================
# CELL 2: Run this to start the interactive tool
# ================================================================
merger.start()